In [ ]:
!pip install sumy
!pip install datasets
!pip install tqdm
!pip install evaluate
!pip install rouge_score

In [ ]:
import os
import json
import nltk
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = '/content/drive/MyDrive/nlp-project/data'
BILLSUM_DATA_PATH = os.path.join(DATA_PATH, 'billsum_v4_1')

In [ ]:
from datasets import load_dataset

In [ ]:
TRAIN_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_train_data_final_OFFICIAL.jsonl")
TEST_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_test_data_final_OFFICIAL.jsonl")

dataset = load_dataset("json", data_files={
    "train": TRAIN_DATA_FILE,
    "test": TEST_DATA_FILE,
})

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0a257b7a118a3705/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from sumy.summarizers.kl import KLSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from tqdm import tqdm
import evaluate

In [ ]:
def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def evaluate_summaries(dataset, metric, batch_size=16, column_text="text", column_summary="summary"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))
    summarizer = KLSummarizer()

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)
    ):
        predicted_sums = []
        for text in article_batch:
            parser = PlaintextParser.from_string(text, Tokenizer("english"))
            predicted = summarizer(parser.document, 5)
            predicted = " ".join(list(map(lambda x: str(x), predicted)))
            predicted_sums.append(predicted)
        metric.add_batch(predictions=predicted_sums, references=target_batch)
    
    score = metric.compute()
    return score

In [ ]:
rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

score = evaluate_summaries(dataset['test'], rouge_metric)

100%|██████████| 205/205 [06:58<00:00,  2.04s/it]


In [ ]:
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"KLSum"])

,rouge1,rouge2,rougeL,rougeLsum
KLSum,0.304996,0.116818,0.195518,0.195593
